##  <center> Пишем своего телеграм-бота :)</center>



In [1]:
import telebot
from telebot import types

In [2]:
# подключим токен нашего бота
bot = telebot.TeleBot("бот")

In [3]:
# напишем, что делать нашему боту при команде старт
@bot.message_handler(commands=['start'])
def send_keyboard(message, text="Привет, чем я могу тебе помочь?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=2)  # наша клавиатура
    itembtn1 = types.KeyboardButton('Добавить дело в список') # создадим кнопку
    itembtn2 = types.KeyboardButton('Показать список дел')
    itembtn3 = types.KeyboardButton('Удалить дело из списка')
    itembtn4 = types.KeyboardButton("Удалить все дела из списка")
    itembtn5 = types.KeyboardButton('Другое')
    itembtn6 = types.KeyboardButton('Пока все!')
    keyboard.add(itembtn1, itembtn2) # добавим кнопки 1 и 2 на первый ряд
    keyboard.add(itembtn3, itembtn4, itembtn5, itembtn6) # добавим кнопки 3, 4, 5 на второй ряд
    # но если кнопок слишком много, они пойдут на след ряд автоматически

    # пришлем это все сообщением и запишем выбранный вариант
    msg = bot.send_message(message.from_user.id,
                     text=text, reply_markup=keyboard)

    # отправим этот вариант в функцию, которая его обработает
    bot.register_next_step_handler(msg, callback_worker)

## Добавляем SQLite

Нам нужно, чтобы для каждого обратившегося к ней пользователя наша система хранила список дел. Я предлагаю воспользоваться базой данных SQLite – она быстрая (т.к. обращается напрямую к файлам, без портов и сокетов), легко встраивается в приложения и проста в использовании. Вы можете попробовать подключить более продвинутые MySQL и PostgreSQL, но это потребует чуть больше времени. 

Можно и просто сохранять все в обычный файлик, но помните, что такой вариант масштабировать не получится – вспомните историю, [как в Великобритании потеряли 16 тысяч положительных тестов на коронавирус.](https://www.kommersant.ru/doc/4520501)

In [4]:
import sqlite3

In [5]:
# подключаем базу данных
conn = sqlite3.connect('planner_hse.db')

# курсор для работы с таблицами
cursor = conn.cursor()

In [6]:
try:
    # sql запрос для создания таблицы
    query = "CREATE TABLE \"planner\" (\"ID\" INTEGER UNIQUE, \"user_id\" INTEGER, \"plan\" TEXT, PRIMARY KEY (\"ID\"))"
    # исполняем его –> ура, теперь у нас есть таблица, куда будем все сохранять!
    cursor.execute(query)
except:
    pass


## Пишем функции нашему боту

В начале занятия мы сделали 6 кнопок для нашего бота:
1. Добавить дело в список
2. Показать список дел
3. Удалить дело из списка
4. Удалить все дела из списка
5. Другое
6. Пока все!

Давайте по очереди напишем функции для каждой из них.

**Добавить дело в список**

In [7]:
# напишем функции для каждого случая
# эта добавляет строчку с планом в хранилище
def add_plan(msg):
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('INSERT INTO planner (user_id, plan) VALUES (?, ?)',
                       (msg.from_user.id, msg.text))
        con.commit()
    bot.send_message(msg.chat.id, 'Запомню :-)')
    send_keyboard(msg, "Чем еще могу помочь?")

**Показать список дел**

In [8]:
# просто функция, которая делает нам красивые строки для отправки пользователю
def get_plans_string(tasks):
    tasks_str = []
    for val in list(enumerate(tasks)): # val=(0, (дело1))
        tasks_str.append(str(val[0] + 1) + ') ' + val[1][0] + '\n')
    return ''.join(tasks_str)

# отправляем пользователю его планы
def show_plans(msg):
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT plan FROM planner WHERE user_id=={}'.format(msg.from_user.id))
        tasks = get_plans_string(cursor.fetchall())
        bot.send_message(msg.chat.id, tasks)
        send_keyboard(msg, "Чем еще могу помочь?")

**Удалить дело из списка**

In [9]:
# выыделяет одно дело, которое пользователь хочет удалить
def delete_one_plan(msg):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        # достаем все задачи пользователя
        cursor.execute('SELECT plan FROM planner WHERE user_id=={}'.format(msg.from_user.id))
        # достанем результат запроса
        tasks = cursor.fetchall()
        for value in tasks:
            markup.add(types.KeyboardButton(value[0]))
        msg = bot.send_message(msg.from_user.id,
                               text = "Выбери одно дело из списка",
                               reply_markup=markup)
        bot.register_next_step_handler(msg, delete_one_plan_)

# удаляет это дело
def delete_one_plan_(msg):
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('DELETE FROM planner WHERE user_id==? AND plan==?', (msg.from_user.id, msg.text))
        bot.send_message(msg.chat.id, 'Ура, минус одна задача!')
        send_keyboard(msg, "Чем еще могу помочь?")

**Удалить все дела из списка**

In [10]:
# удаляет все планы для конкретного пользователя
def delete_all_plans(msg):
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('DELETE FROM planner WHERE user_id=={}'.format(msg.from_user.id))
        con.commit()
    bot.send_message(msg.chat.id, 'Удалены все дела. Хорошего отдыха!')
    send_keyboard(msg, "Чем еще могу помочь?")

In [11]:
# привязываем функции к кнопкам на клавиатуре
def callback_worker(call):
    if call.text == "Добавить дело в список":
        msg = bot.send_message(call.chat.id, 'Давайте добавим дело! Напишите его в чат')
        bot.register_next_step_handler(msg, add_plan)

    elif call.text == "Показать список дел":
        try:
            show_plans(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Можно отдыхать :-)')
            send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Удалить дело из списка":
        try:
            delete_one_plan(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Можно отдыхать :-)')
            send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Удалить все дела из списка":
        try:
            delete_all_plans(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Можно отдыхать :-)')
            send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Другое":
        bot.send_message(call.chat.id, 'Больше я пока ничего не умею :-(')
        send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Пока все!":
        bot.send_message(call.chat.id, 'Хорошего дня! Когда захотите продолжнить нажмите на команду /start')

In [12]:
@bot.message_handler(content_types=['text'])
def handle_docs_audio(message):
    send_keyboard(message, text="Я не понимаю :-( Выберите один из пунктов меню:")

In [ ]:
bot.polling(none_stop=True) # или bot.infinity_polling()

2022-12-08 17:01:20,186 (__init__.py:1083 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
2022-12-08 17:01:20,192 (__init__.py:1085 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "C:\Users\mbbur\anaconda3\lib\site-packages\telebot\__init__.py", line 1073, in __threaded_polling
    polling_thread.raise_exceptions()
  File "C:\Users\mbbur\anaconda3\lib\site-packages\telebot\util.py", line 118, in raise_exceptions
    raise self.exception_info
  File "C:\Users\mbbur\anaconda3\lib\site-packages\telebot\util.py", line 100, in run
    task(*args, **kwargs)
  File "C:\Users\mbbur\anaconda3\lib\site-packages\telebot\__init__.py", line 649, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1),
  File "C:\Users\mbbur\anaconda3\lib\sit